In [1]:
# Read csv
from tqdm import tqdm
from collections import defaultdict
import csv

champions = {}
with open('/Users/tonykim/Downloads/champions-CCs,lanes.csv') as f:
    reader = csv.DictReader(f)
    for champion in tqdm(reader):
        champions[int(champion['key'])] = champion

with open('/Users/tonykim/Downloads/merged_match_dataset_V0.csv') as f:
    reader = csv.DictReader(f)
    rows = [row for row in tqdm(reader)]

146it [00:00, 37110.99it/s]
350880it [00:16, 21344.10it/s]


In [2]:
champion_picks_rows = defaultdict(list) # key -> list
champion_bans_rows = defaultdict(list)

for row in tqdm(rows):
    champion_picks_rows[int(row['championId_Pick'])].append(row)
    champion_bans_rows[int(row['championId_Ban'])].append(row)


100%|██████████| 350880/350880 [00:00<00:00, 746905.51it/s]


In [3]:
def calculate_kda(k, d, a):
    if d == 0:
        return k + a
    return (k + a) / d

In [4]:
for c_id, champion in champions.items():
    # Select rows
    pick_rows = champion_picks_rows[c_id]
    ban_rows = champion_bans_rows[c_id]
    
    # Calculate winrate
    win_rate = sum([1 for row in pick_rows if row['win_x'] == 'Win']) / len(pick_rows)
    print(f'Win rate for champion {champion["name"]} is {win_rate*100:.4f}')
    
    # Calculate pickrate
    # picks = # of filtered_rows
    # total_games = rows/10
    # pick_rate = picks / total_games
    total_games = (len(rows) // 10)
    pick_rate = len(pick_rows) / total_games
    ban_rate = len(ban_rows) / total_games
    pick_ban_rate = pick_rate + ban_rate
    print(f'Pick rate for champion {champion["name"]} is {pick_rate*100:.04f}')
    print(f'Ban rate for champion {champion["name"]} is {ban_rate*100:.04f}')
    print(f'PickBan rate is {pick_ban_rate*100:.04f}')
    
    # Calculate KDA
    kda = calculate_kda(sum(int(row['kills']) for row in pick_rows), sum(int(row['deaths']) for row in pick_rows), sum(int(row['deaths']) for row in pick_rows))
    print(f'KDA for champion {champion["name"]} is {kda:.04f}')
    
    # Store to champions dataset
    champion['win_rate'] = win_rate
    champion['pick_rate'] = pick_rate
    champion['ban_rate'] = ban_rate
    champion['pick_ban_rate'] = pick_ban_rate
    champion['average_KDA'] = kda

Win rate for champion Aatrox is 45.4060
Pick rate for champion Aatrox is 5.3352
Ban rate for champion Aatrox is 5.3409
PickBan rate is 10.6760
KDA for champion Aatrox is 1.9056
Win rate for champion Ahri is 51.3546
Pick rate for champion Ahri is 9.3622
Ban rate for champion Ahri is 5.5717
PickBan rate is 14.9339
KDA for champion Ahri is 2.1663
Win rate for champion Akali is 45.6451
Pick rate for champion Akali is 10.4708
Ban rate for champion Akali is 19.3143
PickBan rate is 29.7851
KDA for champion Akali is 2.2134
Win rate for champion Alistar is 45.5263
Pick rate for champion Alistar is 3.2490
Ban rate for champion Alistar is 0.9234
PickBan rate is 4.1724
KDA for champion Alistar is 1.3223
Win rate for champion Amumu is 50.1918
Pick rate for champion Amumu is 4.4574
Ban rate for champion Amumu is 0.8065
PickBan rate is 5.2639
KDA for champion Amumu is 1.8340
Win rate for champion Anivia is 51.2338
Pick rate for champion Anivia is 2.4253
Ban rate for champion Anivia is 1.1400
PickBan 

KDA for champion Morgana is 1.5342
Win rate for champion Nami is 51.5829
Pick rate for champion Nami is 9.8125
Ban rate for champion Nami is 0.6555
PickBan rate is 10.4680
KDA for champion Nami is 1.3639
Win rate for champion Nasus is 50.1160
Pick rate for champion Nasus is 6.1417
Ban rate for champion Nasus is 4.7538
PickBan rate is 10.8955
KDA for champion Nasus is 1.9068
Win rate for champion Nautilus is 51.7019
Pick rate for champion Nautilus is 16.3275
Ban rate for champion Nautilus is 29.8535
PickBan rate is 46.1810
KDA for champion Nautilus is 1.4584
Win rate for champion Neeko is 49.3485
Pick rate for champion Neeko is 3.4998
Ban rate for champion Neeko is 0.6783
PickBan rate is 4.1781
KDA for champion Neeko is 1.8659
Win rate for champion Nidalee is 46.0411
Pick rate for champion Nidalee is 2.9155
Ban rate for champion Nidalee is 0.2593
PickBan rate is 3.1749
KDA for champion Nidalee is 2.2144
Win rate for champion Nocturne is 51.1514
Pick rate for champion Nocturne is 7.5496


In [5]:
# Sanity check
sum(champion['pick_rate'] for champion in champions.values()) # Should be 10.0

10.0

In [6]:
# Save results
with open('champions_rates_V0.csv', 'w') as output:
    writer = csv.DictWriter(output, fieldnames=champions[266].keys())
    writer.writeheader()
    for champion in champions.values():
        writer.writerow(champion)